# ETL PROJECT
##### Lauren Chavez, James DeCola 

### Part 1 - Importing Requirements

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np

#You may need to update config.py in this folder if your connection string and mine are different!
from config import connect_string
print(f'CONNECTION STRING: {connect_string}')

CONNECTION STRING: postgres:p4ssword@localhost:5432


### Part 2 - Extracting and Transforming 

#### Natural Disaster Event counts and Economic Impact

In [2]:
#Load economic damage csv
csv_file = "./Resources/economic-damage-from-natural-disasters.csv"
econ_damage_df = pd.read_csv(csv_file)
econ_damage_df.head()

,Entity,Code,Year,Total economic damage from natural disasters (US$)
0,All natural disasters,NaN,1900,30000000
1,All natural disasters,NaN,1901,0
2,All natural disasters,NaN,1902,0
3,All natural disasters,NaN,1903,480000000
4,All natural disasters,NaN,1904,0


In [3]:
#cleaning up economic damage frame
econ_damage_df.rename(columns={"Entity": "disaster_type", "Year": "year", "Total economic damage from natural disasters (US$)":"econ_damage"}, inplace=True)
econ_damage_df = econ_damage_df[[ "year", "disaster_type", "econ_damage"]]
econ_damage_df.head()

,year,disaster_type,econ_damage
0,1900,All natural disasters,30000000
1,1901,All natural disasters,0
2,1902,All natural disasters,0
3,1903,All natural disasters,480000000
4,1904,All natural disasters,0


In [4]:
#Load disaster count csv
csv_file = "./Resources/number-of-natural-disaster-events.csv"
event_count_df = pd.read_csv(csv_file)
event_count_df.head()

,Entity,Code,Year,Number of reported natural disasters (reported disasters)
0,All natural disasters,NaN,1900,5
1,All natural disasters,NaN,1901,2
2,All natural disasters,NaN,1902,9
3,All natural disasters,NaN,1903,8
4,All natural disasters,NaN,1904,2


In [5]:
#cleaning up disaster count frame
event_count_df.rename(columns={"Entity": "disaster_type", "Year": "year", "Number of reported natural disasters (reported disasters)":"disaster_count"}, inplace=True)
event_count_df = event_count_df[[ "year", "disaster_type", "disaster_count"]]
event_count_df.head()

,year,disaster_type,disaster_count
0,1900,All natural disasters,5
1,1901,All natural disasters,2
2,1902,All natural disasters,9
3,1903,All natural disasters,8
4,1904,All natural disasters,2


In [6]:
#joining frames into one useful frame.
#Not as familiar with pd.merge, but spotchecks showed that ouput 
#was what I'm looking for, and row count is right!

disaster_count_econ = pd.merge(event_count_df, econ_damage_df,  how='left', left_on=['year','disaster_type'], right_on = ['year','disaster_type'])
disaster_count_econ.head()

,year,disaster_type,disaster_count,econ_damage
0,1900,All natural disasters,5,30000000.0
1,1901,All natural disasters,2,0.0
2,1902,All natural disasters,9,0.0
3,1903,All natural disasters,8,480000000.0
4,1904,All natural disasters,2,0.0


In [7]:
#Creating a df that is merged and separated by disaster type - now it starts looking like a relational db table!

disaster_df = disaster_count_econ.pivot_table(['disaster_count','econ_damage'],['year'],'disaster_type')

disaster_df.columns = ['all_disaster_count','drought_count','earthquake_count','extreme_temp_count','extreme_weather_count','flood_count','impact_count','landslide_count','dry_mass_movement_count','volcanic_count','wildfire_count',
                      'all_disaster_cost','drought_cost','earthquake_cost','extreme_temp_cost','extreme_weather_cost','flood_cost','impact_cost','landslide_cost','dry_mass_movement_cost','volcanic_cost','wildfire_cost']

disaster_df= disaster_df.reset_index()
disaster_df = disaster_df.fillna(0)
disaster_df.head(10)

,year,all_disaster_count,drought_count,earthquake_count,extreme_temp_count,extreme_weather_count,flood_count,impact_count,landslide_count,dry_mass_movement_count,...,drought_cost,earthquake_cost,extreme_temp_cost,extreme_weather_cost,flood_cost,impact_cost,landslide_cost,dry_mass_movement_cost,volcanic_cost,wildfire_cost
0,1900,5.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,30000000.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1901,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1902,9.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1903,8.0,1.0,1.0,0.0,2.0,2.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,480000000.0,0.0,0.0,0.0,0.0,0.0
4,1904,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1905,4.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1906,17.0,1.0,10.0,0.0,3.0,2.0,0.0,0.0,0.0,...,0.0,630750000.0,0.0,20000000.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1907,5.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,30000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1908,4.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,116000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1909,11.0,0.0,3.0,0.0,5.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Global Temperatures 

In [8]:
#Load global temps csv file
csv_file = "./Resources/GlobalTemperatures.csv"
yearly_temps_df = pd.read_csv(csv_file)
yearly_temps_df.head()

,dt,LandAverageTemperature,LandAverageTemperatureUncertainty,LandMaxTemperature,LandMaxTemperatureUncertainty,LandMinTemperature,LandMinTemperatureUncertainty,LandAndOceanAverageTemperature,LandAndOceanAverageTemperatureUncertainty
0,1750-01-01,3.034,3.574,NaN,NaN,NaN,NaN,NaN,NaN
1,1750-02-01,3.083,3.702,NaN,NaN,NaN,NaN,NaN,NaN
2,1750-03-01,5.626,3.076,NaN,NaN,NaN,NaN,NaN,NaN
3,1750-04-01,8.490,2.451,NaN,NaN,NaN,NaN,NaN,NaN
4,1750-05-01,11.573,2.072,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#converting dt into a date, removing all data before our scope.
yearly_temps_df['dt'] = pd.to_datetime(yearly_temps_df['dt'])
yearly_temps_df['year'] = pd.DatetimeIndex(yearly_temps_df['dt']).year
yearly_temps_df = yearly_temps_df.loc[yearly_temps_df['dt'] >= '1900']

#removing columns outside of our scope, some renaming
yearly_temps_df.rename(columns={"LandAndOceanAverageTemperature":"avg_temp"}, inplace=True)
yearly_temps_df = yearly_temps_df[['year','avg_temp']]
yearly_temps_df.head()

,year,avg_temp
1800,1900,13.142
1801,1900,13.777
1802,1900,14.400
1803,1900,15.170
1804,1900,15.955


In [10]:
#grouping by year and taking average.  
grouped_temps = yearly_temps_df.groupby(['year'])
yearly_temps_avg = grouped_temps["avg_temp"].mean()
yearly_temps_cleaned = pd.DataFrame({"avg_temp": yearly_temps_avg})

#resetting index and displaying.
yearly_temps_cleaned = yearly_temps_cleaned.reset_index()
yearly_temps_cleaned.head()

,year,avg_temp
0,1900,15.143917
1,1901,15.073333
2,1902,14.958333
3,1903,14.836583
4,1904,14.810417


#### Lauren's tables!

### Part 3 - Loading

In [11]:
#connecting to db
engine = create_engine(f'postgresql://{connect_string}/disaster_db')
engine.table_names()

['disasters', 'temps']

In [12]:
#Loading db
disaster_df.to_sql(name='disasters', con=engine, if_exists='append', index=False)
yearly_temps_cleaned.to_sql(name='temps', con=engine, if_exists='append', index=False)
#add other db loads here!

In [13]:
#verifying data is in all tables
pd.read_sql_query('select temps.year, temps.avg_temp, disasters.all_disaster_count from temps join disasters on disasters.year = temps.year where temps.year = 1942', con=engine).head(10)

,year,avg_temp,all_disaster_count
0,1942,15.325333,9
